In [288]:
import numpy as np
import jax
import jax.numpy as jnp
import optax
import pulp
import importlib
import matplotlib.pyplot as plt

In [289]:
import GGM_sampler
import GGM_inference
import GGM_diagnostics
import J_sampler
import J_inference
import J_diagnostics

In [290]:
importlib.reload(GGM_inference)
importlib.reload(GGM_diagnostics)
importlib.reload(GGM_sampler)
importlib.reload(J_inference)
importlib.reload(J_diagnostics)
importlib.reload(J_sampler)

<module 'J_sampler' from '/Users/lucaraffo/Desktop/final/J_sampler.py'>

In [285]:
n_spins = 10
n_samples = 1000

In [286]:
J_precision = GGM_sampler.precision_maker(n_spins, seed = 9)

In [ ]:
samples = GGM_sampler.